<h5> test of download_model_function </h5>


In [3]:
from work_with_model import download_and_save_model
model = "sentence-transformers/paraphrase-albert-small-v2"

model.replace("@","@")

'sentence-transformers/paraphrase-albert-small-v2'

In [2]:
from work_with_model import load_model_bin

model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"
# model_name=model.split("/")[-1]
print(model_name)
obj = load_model_bin(model_name)

sentence-transformers/paraphrase-MiniLM-L3-v2


In [3]:
obj[1]

BertTokenizerFast(name_or_path='models/sentence-transformers/paraphrase-MiniLM-L3-v2/tokenizer', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [2]:
from work_with_model import load_model_bin

load_model_bin()

False


(BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 384, padding_idx=0)
       (position_embeddings): Embedding(512, 384)
       (token_type_embeddings): Embedding(2, 384)
       (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-2): 3 x BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=384, out_features=384, bias=True)
               (key): Linear(in_features=384, out_features=384, bias=True)
               (value): Linear(in_features=384, out_features=384, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=384, out_features=384, bias=True)
               (LayerNorm): LayerNor

In [10]:
from work_with_model import encode_model_bin
import torch

def encode_text(text):
    model,tokenizer = load_model_bin()
    print(model is None)

    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=False,
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model(**encoding)
        last_hidden_state = outputs.hidden_states[-1]
        embeddings = last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()


In [12]:
encode_text("nlp book")

False


TypeError: 'NoneType' object is not subscriptable

In [6]:
from dataclasses import dataclass

@dataclass
class model:
    a = 1
    b = a+1

p = model()

In [8]:
p.b

2

## new test

In [3]:
from work_with_model import transformer_ops as tops
name = "GPT_125M"
wom = tops(name=name)
# name = "sentence-transformers@paraphrase-albert-small-v2"

In [5]:
wom.setter()

Directory already exists


In [6]:
wom.download_and_save_model_pickle(name)

GPT_125M already exists no need to download again ...


In [9]:
vec = wom.encode_single_doc("the cat is strong ")

In [10]:
vec

array([-1.11646667e-01, -3.32449190e-02,  4.19502817e-02, -1.35372598e-02,
        1.25394072e-02, -5.47454767e-02, -9.21722874e-03,  1.30647756e-02,
        2.34391186e-02, -1.68136824e-02, -1.19445743e-02,  8.12968761e-02,
       -1.99614093e-02,  5.88950329e-02,  8.64244904e-03,  1.01656780e-01,
        1.35027573e-01, -3.15942317e-02, -4.71081622e-02,  7.98866674e-02,
        3.33311632e-02,  3.95192169e-02,  7.63263740e-03, -9.72219463e-03,
        6.06915988e-02,  1.23816967e-01, -1.88962836e-02,  4.94065601e-03,
        6.64008688e-03,  6.44846410e-02,  1.70861203e-02,  7.19634220e-02,
        3.16454098e-04,  6.44286796e-02, -5.19151688e-02, -6.88865557e-02,
       -1.06475890e-01,  1.02746882e-01, -8.52695294e-03,  4.72451607e-03,
        5.18697612e-02,  1.17117107e-01, -4.46294509e-02,  6.99138418e-02,
        1.90051999e-02, -6.65071160e-02, -3.49321105e-02, -7.01009631e-02,
        3.19888331e-02, -5.05961142e-02, -1.00078635e-01,  3.10247745e-02,
        1.32996336e-01, -

In [1]:
from work_with_model import transformer_ops as tops
name = "GPT_125M"
 
wom = tops(name=name)
wom.setter()

Directory already exists


In [2]:
tokenizer,model = wom.access_model_for_tesing()

AttributeError: 'transformer_ops' object has no attribute 'access_model_for_tesing'

In [17]:
# from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']
sentences = ["the cat is strong "]


# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-albert-small-v2')
# model = AutoModel.from_pretrained('sentence-transformers/paraphrase-albert-small-v2')


def encode_into_vector(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    print("Sentence embeddings:")
    print(sentence_embeddings)
    print(len(sentence_embeddings[0]))

encode_into_vector(sentences)

RuntimeError: expand(torch.LongTensor{[1, 6, 1]}, size=[1, 2]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)